In [37]:
import pandas as pd

btc_cad_path = 'C:/Users/lynst/Documents/GitHub/machine-learning/machine-learning/BTC_CAD.csv'
btc_cad = pd.read_csv('C:/Users/lynst/Documents/GitHub/machine-learning/machine-learning/BTC_CAD.csv')

# Creating a new variable name from the existing dataframe
bitcoin = btc_cad

Each column represents a different feature, for example 'High', 'Low' and 'Close' prices are all features. These features are also known as attributes, input, or predictor variables which can be used to ascertain a stock price prediction (called labels, output, or target variables).

In [38]:
# get stock info
print(bitcoin.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       366 non-null    object 
 1   Open       362 non-null    float64
 2   High       362 non-null    float64
 3   Low        362 non-null    float64
 4   Close      362 non-null    float64
 5   Adj Close  362 non-null    float64
 6   Volume     362 non-null    float64
dtypes: float64(6), object(1)
memory usage: 20.1+ KB
None


The following method gives a breakdown of the first and last entries in the dataset.

In [39]:
bitcoin.info

<bound method DataFrame.info of            Date          Open          High           Low         Close  \
0    2020-04-21   9718.074219   9838.333008   9728.248047   9763.935547   
1    2020-04-22   9762.682617  10125.870120   9747.394531  10096.278320   
2    2020-04-23  10102.092770  10533.734380  10009.763670  10451.163090   
3    2020-04-24  10457.432620  10678.707030  10457.432620  10642.806640   
4    2020-04-25  10642.216800  10773.176760  10601.609380  10669.635740   
..          ...           ...           ...           ...           ...   
361  2021-04-17  76964.703130  78268.406250  75503.000000  75906.359380   
362  2021-04-18  75928.953130  76373.718750  66081.828130  70374.906250   
363  2021-04-19  70344.109380  71803.609380  68107.265630  69788.234380   
364  2021-04-20           NaN           NaN           NaN           NaN   
365  2021-04-21  71509.304690  71556.757810  71132.078130  71477.757810   

        Adj Close        Volume  
0     9763.935547  4.624843e+10  

To view some of the summary statistics I will use the describe() method.

In [40]:
bitcoin.describe()

,Open,High,Low,Close,Adj Close,Volume
count,362.000000,362.000000,362.000000,362.000000,362.000000,3.620000e+02
mean,29707.219509,30555.675634,28874.214073,29864.750594,29864.750594,5.364486e+10
std,21773.554061,22482.286397,20992.767955,21845.825379,21845.825379,3.296501e+10
min,9718.074219,9838.333008,9728.248047,9763.935547,9763.935547,1.663903e+10
25%,13373.918455,13684.485110,13056.077148,13421.041262,13421.041262,3.142625e+10
50%,16520.156250,17169.826175,16253.259275,16921.556640,16921.556640,4.716901e+10
75%,44679.113283,46686.911130,41998.325195,45279.527345,45279.527345,6.929678e+10
max,79623.851560,81445.671880,77726.054690,79598.406250,79598.406250,4.470700e+11


So I can tell the average closing price is C$29,865 to the nearest integer. 

There isn't much data to work from (one years worth exactly), but I would like to create a dataframe object from the price data and try and use it to forecast future price information.

# Import Libraries
The bulk of the libraries I will use are as follows. This may change as I progress.

In [41]:
import numpy as np
from sklearn import linear_model
from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

Taking a look at which columns are present in the dataset gives me:

In [42]:
bitcoin.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

And now lets see the first and last 5 entries in the table:

In [43]:
bitcoin.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-04-21,9718.074219,9838.333008,9728.248047,9763.935547,9763.935547,4.624843e+10
1,2020-04-22,9762.682617,10125.870120,9747.394531,10096.278320,10096.278320,4.716635e+10
2,2020-04-23,10102.092770,10533.734380,10009.763670,10451.163090,10451.163090,6.119120e+10
3,2020-04-24,10457.432620,10678.707030,10457.432620,10642.806640,10642.806640,4.881932e+10
4,2020-04-25,10642.216800,10773.176760,10601.609380,10669.635740,10669.635740,4.643028e+10


And now the tail of the dataframe.

In [44]:
bitcoin.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
361,2021-04-17,76964.70313,78268.40625,75503.00000,75906.35938,75906.35938,8.272967e+10
362,2021-04-18,75928.95313,76373.71875,66081.82813,70374.90625,70374.90625,1.220180e+11
363,2021-04-19,70344.10938,71803.60938,68107.26563,69788.23438,69788.23438,8.183693e+10
364,2021-04-20,NaN,NaN,NaN,NaN,NaN,NaN
365,2021-04-21,71509.30469,71556.75781,71132.07813,71477.75781,71477.75781,8.495634e+10


# Cleaning the Data
A brief look at the summary info for the dataset tells me there are a different number of date values compared to the rest of the column entries. This will need to be addressed, followed by any missing values, NaN, 'isnull' or entries with 'zero'. I can also see the different number 'count' for entries using the count() method:

In [45]:
bitcoin.count()

Date         366
Open         362
High         362
Low          362
Close        362
Adj Close    362
Volume       362
dtype: int64

First I just want to see what shape the dataframe is.

In [46]:
bitcoin.shape

(366, 7)

So it's 366 rows by 7 columns or a (366x7) matrix.

Next, to find the total number of missing entries.

In [48]:
bitcoin_missing_values_count = bitcoin.isnull().sum()
bitcoin_missing_values_count[0:7]

Date         0
Open         4
High         4
Low          4
Close        4
Adj Close    4
Volume       4
dtype: int64

In [49]:
total_bitcoin = np.product(bitcoin.shape)
total_missing_bitcoin = bitcoin_missing_values_count.sum()

percent_missing_values = (total_missing_bitcoin/total_bitcoin) * 100
print(percent_missing_values)

0.936768149882904


From this I can see that just under 1% of the total values are missing. I can either choose to leave them out completely (as they may not really affect my model for the purpose of this exercise) or use imputed values such as the mean or median.

Having checked the csv file I can determine that there is no real reason for the missing values (null entries) for these dates. At first I thought these missing values might be due to public holidays but because Bitcoin is a currency, it appears to be traded OTC (Over The Counter) beyong normal trading hours throughout the day, seven days a week. As a result I have chosen to leave these missing values out using the dropna() method to eliminate them.

In [50]:
bitcoin.dropna()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-04-21,9718.074219,9838.333008,9728.248047,9763.935547,9763.935547,4.624843e+10
1,2020-04-22,9762.682617,10125.870120,9747.394531,10096.278320,10096.278320,4.716635e+10
2,2020-04-23,10102.092770,10533.734380,10009.763670,10451.163090,10451.163090,6.119120e+10
3,2020-04-24,10457.432620,10678.707030,10457.432620,10642.806640,10642.806640,4.881932e+10
4,2020-04-25,10642.216800,10773.176760,10601.609380,10669.635740,10669.635740,4.643028e+10
...,...,...,...,...,...,...,...
360,2021-04-16,79368.234380,79831.468750,75262.203130,77018.320310,77018.320310,1.054380e+11
361,2021-04-17,76964.703130,78268.406250,75503.000000,75906.359380,75906.359380,8.272967e+10
362,2021-04-18,75928.953130,76373.718750,66081.828130,70374.906250,70374.906250,1.220180e+11
363,2021-04-19,70344.109380,71803.609380,68107.265630,69788.234380,69788.234380,8.183693e+10


So, to begin I will start the Exploratory Data Analysis phase. The mean value for the Bitcoin (CAD) close price is C$ 29,864.75. From the summary statistics I can also see the value has been lingering in the upper-quartile range for the last year and a half roughly.

As there are no dividends or stock splits because Bitcoin is a currency (medium of exchange / store of value), only the first 6 columns are of any use to me in this particular dataset: Date, Open, High, Low, Close and Volume.

To view the first 5 five entries is the same as using the head() method.

In [17]:
bitcoin.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'], dtype='object')

In [18]:
bitcoin = bitcoin.dropna(axis=0)

Next I want to split the data into predictors and a target variable, containing all my feature columns in one table variable and the target variable in a column vector. The prediction target can be assigned as follows:

In [19]:
y = bitcoin.Close

The features are a selection of columns used to predict 'y', including

In [21]:
bitcoin_features = ['Date','Open','High','Low','Volume']

In [22]:
X = bitcoin[bitcoin_features]

KeyError: "['Date'] not in index"

In [14]:
print(bitcoin[:5])

                    Open          High           Low         Close  \
Date                                                                 
2019-09-18  13570.838867  13664.313477  13552.004883  13561.018555   
2019-09-19  13564.007812  13649.998047  13061.373047  13611.212891   
2019-09-20  13611.084961  13642.738281  13438.896484  13504.491211   
2019-09-21  13507.153320  13513.054688  13264.508789  13289.721680   
2019-09-22  13295.555664  13368.072266  13166.497070  13362.695312   

                 Volume  Dividends  Stock Splits  
Date                                              
2019-09-18  21500923601          0             0  
2019-09-19  26433391055          0             0  
2019-09-20  19542795567          0             0  
2019-09-21  17806696605          0             0  
2019-09-22  17515000629          0             0  


In [16]:
# the last 5 or most recent prices are the same as using the tail() method
print(bitcoin[-5:])

                    Open          High           Low         Close  \
Date                                                                 
2021-04-16  79368.234375  79831.468750  75262.203125  77018.320312   
2021-04-17  76964.703125  78268.406250  75503.000000  75906.359375   
2021-04-18  75928.953125  76373.718750  66081.828125  70374.906250   
2021-04-19  70344.109375  71803.609375  68107.265625  69788.234375   
2021-04-21  69859.906250  71937.265625  66774.476562  71087.851562   

                  Volume  Dividends  Stock Splits  
Date                                               
2021-04-16  105437908391          0             0  
2021-04-17   82729666943          0             0  
2021-04-18  122017577989          0             0  
2021-04-19   81836928952          0             0  
2021-04-21   85922406400          0             0  


So currently I can determine that Bitcoin was trading well below the mean (19 months ago) and now it's trading at roughly 250% above the mean for the last 19 month period. Time for some interval analysis where I will attempt to place OHLC values in different interval buckets based on the number of counts.

In [ ]:
''''''def get_counts(sequence):
    df.loc[(df['Open'] > 5000) & (df['Open'] < 10000)]
    df.loc[(df['Open'] > 10000) & (df['Open'] < 20000)]
    df.loc[(df['Open'] > 20000) & (df['Open'] < 30000)]
    df.loc[(df['Open'] > 30000) & (df['Open'] < 40000)]
    df.loc[(df['Open'] > 40000) & (df['Open'] < 50000)]
    df.loc[(df['Open'] > 50000) & (df['Open'] < 60000)]
    df.loc[(df['Open'] > 60000) & (df['Open'] < 70000)]
    df.loc[(df['Open'] > 70000) & (df['Open'] < 80000)]